In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import control as ctrl
import matplotlib.pyplot as plt
import scipy as sp
import scipy.linalg as la

Considers a system 

$$\ddot{x} + 2 \dot{x} + 100 x = f(t)$$
with a displacement sensor. 

The state space representation is 

$$\dot{\mathbf{z}} = A \mathbf{z} + B u$$

$$y = C \mathbf{z} + D u$$ 

where 

$$A = 
\begin{bmatrix}
0&1\\
-100&-2
\end{bmatrix}
,\quad
B = 
\begin{bmatrix}0\\1\end{bmatrix}
,\quad
C = \begin{bmatrix}1&0\end{bmatrix}
, \text{ and }
D = [0]$$

In [2]:
sample_freq = 1e3
A = sp.array([[0, 1],\
              [-100, 2]])
B = sp.array([[0], [1]])
C = sp.array([[1, 0]])
D = sp.array([[0]])
sys = ctrl.ss(A, B, C, D)


For this example, we will generate an impulse response for only 4 discrete times.

In [56]:
t = sp.linspace(0, 0.01, num = 6, endpoint = False)
dt = t[1]
t

array([ 0.        ,  0.00166667,  0.00333333,  0.005     ,  0.00666667,
        0.00833333])

The impulse response for a second order underdamped system is known to be

$$h(t) = \frac{1}{\omega_d}e^{-\zeta \omega_n t}\sin\left(\omega_d t\right)$$

In [5]:
omega_n = sp.sqrt(100)
zeta = 2/(2*omega_n)
omega_d = omega_n * sp.sqrt(1 - zeta**2)

In [14]:
h = 1/omega_d * sp.exp(-zeta*omega_n*t)*sp.sin(omega_d*t)
h

array([ 0.        ,  0.00166381,  0.00332163,  0.00497301,  0.00661751,
        0.00825471])

In [ ]:
plt.plot(t.T,h.T)

In [15]:
Hankel_0 = sp.vstack((h[0:-2],h[1:-1]))
Hankel_0 = Hankel_0.T
Hankel_0

array([[ 0.        ,  0.00166381],
       [ 0.00166381,  0.00332163],
       [ 0.00332163,  0.00497301],
       [ 0.00497301,  0.00661751]])

In [17]:
h[1:-1]

array([ 0.00166381,  0.00332163,  0.00497301,  0.00661751])

In [19]:
h[2:]

array([ 0.00332163,  0.00497301,  0.00661751,  0.00825471])

In [20]:
Hankel_1 = sp.vstack((h[1:-1],h[2:]))
Hankel_1 = Hankel_1.T
Hankel_1

array([[ 0.00166381,  0.00332163],
       [ 0.00332163,  0.00497301],
       [ 0.00497301,  0.00661751],
       [ 0.00661751,  0.00825471]])

In [34]:
U, sig, Vt = la.svd(Hankel_0)
V = Vt.T
U = U[:,:2]
print(U)
V = V[:,:2]
print(V)

[[-0.12593212 -0.8276856 ]
 [-0.33679427 -0.43164131]
 [-0.54686132 -0.03679528]
 [-0.75607766  0.356747  ]]
[[-0.56118673  0.82768923]
 [-0.82768923 -0.56118673]]


In [35]:
sig = sp.diag(sig)
print(sig)

[[ 0.01093543  0.        ]
 [ 0.          0.0011281 ]]


In [38]:
A_d = la.inv(sp.sqrt(sig))@U.T@Hankel_1@V@la.inv(sp.sqrt(sig))
print(A_d)

[[ 1.36921489 -0.22308024]
 [ 0.61828873  0.62718002]]


In [58]:
lam_d, vec = la.eig(A_d)
print(lam_d)
print(vec)

[ 0.99819745+0.01655475j  0.99819745-0.01655475j]
[[ 0.51440527+0.0229527j  0.51440527-0.0229527j]
 [ 0.85723998+0.j         0.85723998-0.j       ]]


In [59]:
# This should be the same as A_d
print(A_d)
print(vec@sp.diag(lam)@la.inv(vec))

[[ 1.36921489 -0.22308024]
 [ 0.61828873  0.62718002]]
[[ 1.36921489 +3.55271368e-15j -0.22308024 -1.77635684e-15j]
 [ 0.61828873 +0.00000000e+00j  0.62718002 -1.77635684e-15j]]


In [61]:
lam = sp.log(lam_d)/dt
lam

array([-1.+9.94987437j, -1.-9.94987437j])

In [64]:
# These are the continuous time eigenvalues
print('The undamped natural frequency is {} rad/sec.'.format(abs(lam[0])))
print('The undamped natural frequency is {}.'.format(-sp.real(lam[0])/abs(lam[0])))


The undamped natural frequency is 10.000000000000185 rad/sec.
The undamped natural frequency is 0.10000000000002313 rad/sec.
